# REL 3: How does your system adapt to changes in demand?

Resources provisioning managed by the service or automated. The goal of resource management is to consume the minimum resources to keep the application running reliably, but no less than that. Scaling resources, such as EC2 Instances or Kinesis Streams, requires a single action but needs understanding of the system dynamics to balance between provisioning too little, and causing bottlenecks, or too much, causing waste. 

The level of resource management is different in each service. Some developers just want to execute some code in response to events, such as HTTP requests. AWS Lambda provides that and automatically manages containers, instances, network interfaces and so on. Others want to run the container, but not manage resources, and that is provided by AWS Fargate. One layer down, Amazon ECS lets you not only run, but manage container clusters in detail. And so on until reach the individual compute instances on Amazon EC2, even reaching bare metal if desired.

It pays off to climb this abstraction ladder. The pricing structure of the services is built so that it is attractive to let Amazon do the heavy lifting of resource management and you worry about your business logic. Scaling will be different in each context, but in the absence of further requirements, prefer implementing in the highest resource management level type:

* Serverless became the popular jargon for all services that require little to none provisioning and management of infrastructure, like AWS Lambda, Amazon API Gateway and Amazon Cognito. There are still servers of course, just not under customer management. As explained by AWS Serverless Developer Advocate Paul Johnston: “’Serverless’ is just a name. We could have called it ‘Jeff’”.

* Scheduled Scaling is very simple to setup and can be sufficient for the applications where variation in demand is predictable. If the users arrive in the morning and leave in the night or if there is that big holiday or news coming up, just let the schedule handle it. Auto Scaling Scheduled Action, CloudWatch Events and Elastic Beanstalk Scheduled Scaling can help you to program actions without managing your own task server software, such as cron.

* Auto Scaling refers to adjusting provisioning in reaction to CloudWatch Alarms, binding an action to a metric statistic threshold. One simple example, suitable for many applications, would be “Add 1 server when the average CPU Utilization is above 80% for 5 minutes” and “Remove 1 server when the maximum memory utilization is 10% for 10 minutes”. The provisioning actions are executed when the alarm is triggered so applications can scale to unexpected fluctuations in demand. CloudWatch aggregates data in time using statistics like minimum, maximum, count, samples and average. When using the average statistic, it is always important to consider the metric variance. When the metric vary too much, as usual in small environments, the average metric may not be stable enough for auto-scaling.

* Predictive Scaling can be used to have the best of scheduled and auto scaling. For some applications it may be difficult to define the scaling rules as they become too many and possibly conflicting, for example the CPU metric may be too high while the memory metric is too low. For component that takes more time to be provisioned, more advance is needed in the request so that demand does not trample capacity. It may be helpful to train a machine learning model with actual provisioning history and get anticipated scaling suggestions. This is feature is not natively provided by AWS, but pioneered by Netflix project Scryer and growing in implementations and possibilities.

Realistically, applications are architected composing the best resource management for each component. For example, delivery of the static files and simple computations can be totally serverless, while database storage may use auto scaling and some components may not even be horizontally scalable at all. 

Vertical scaling is anticipated and monitored. Relational database writes, directories, resource managers and other single-master services frequently requires vertical scaling. When load can’t be balanced, switching to a larger server may need to be scheduled for downtime risk. As this can take more time, it requires more margin in provisioning for keeping up with future demand before replacement. 

Capacity tests are executed periodically. However well code behaves on development, it may be surprising in production. Even a simple change in a regular expression or SQL query can become a performance bottleneck, not only in your code but in any of its dependencies and libraries. Not only the code changes unpredictably, but CPU and network allocation and other limitations may fluctuate randomly. As the components of performance are so unpredictable, the only way to assess the resulting system scalability is testing it from the outside.

* Be thorough but not exhaustive
* Replay production traffic
* Learn from outliers (percentiles)
* Provision according to variance

Just like most types of tests, it is not viable to be exhaustive but it pays off to be thorough. It may be impossible to stimulate all invocation paths, but testing the most important and common use cases frequently is a important step for reliability and not hard to implement. AWS does not currently offer a load testing service or tool, but you can easily integrate open source tools into Code Pipeline. 
